<a href="https://colab.research.google.com/github/proffranciscofernando/titanic_survival_prediction/blob/main/main_pt_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline de Modelos de Machine Learning com Scikit-Learn e Otimização de Hiperparâmetros

Neste notebook, vamos explorar como construir um **pipeline de machine learning** utilizando a biblioteca **Scikit-Learn**. Abordaremos desde o pré-processamento dos dados até a otimização de múltiplos modelos de classificação com **Grid Search** e validação cruzada. Por fim, implementaremos uma interface interativa para que usuários possam inserir novos dados e obter previsões de forma iterativa.

## Sumário

1. [Objetivos](#1-objetivos)
2. [Importando as Bibliotecas Necessárias](#2-importando-as-bibliotecas-necessárias)
3. [Carregando e Explorando o Conjunto de Dados](#3-carregando-e-explorando-o-conjunto-de-dados)
   - [3.1 Carregando o Dataset](#31-carregando-o-dataset)
   - [3.2 Análise Exploratória de Dados](#32-análise-exploratória-de-dados)
   - [3.3 Selecionando Recursos e Variável Alvo](#33-selecionando-recursos-e-variável-alvo)
   - [3.4 Tratamento de Valores Faltantes](#34-tratamento-de-valores-faltantes)
4. [Preparação dos Dados](#4-preparação-dos-dados)
   - [4.1 Dividindo os Dados em Treino e Teste](#41-dividindo-os-dados-em-treino-e-teste)
5. [Construindo o Pipeline](#5-construindo-o-pipeline)
   - [5.1 Criando o Pipeline de Pré-processamento](#51-criando-o-pipeline-de-pré-processamento)
6. [Otimização de Hiperparâmetros com Grid Search](#6-otimização-de-hiperparâmetros-com-grid-search)
   - [6.1 Definindo os Modelos e Parâmetros](#61-definindo-os-modelos-e-parâmetros)
   - [6.2 Executando o Grid Search com Validação Cruzada](#62-executando-o-grid-search-com-validação-cruzada)
   - [6.3 Comparando os Modelos Otimizados](#63-comparando-os-modelos-otimizados)
7. [Selecionando e Salvando o Melhor Modelo](#7-selecionando-e-salvando-o-melhor-modelo)
8. [Carregando o Modelo e Fazendo Previsões](#8-carregando-o-modelo-e-fazendo-previsões)
   - [8.1 Fazendo Previsões com Novos Dados do Usuário](#81-fazendo-previsões-com-novos-dados-do-usuário)
9. [Conclusão](#9-conclusão)
10. [Referências](#10-referências)


## 1. Objetivos

- **Entender o conceito de pipeline em machine learning.**
- **Implementar um pipeline que inclua etapas de pré-processamento e múltiplos modelos de classificação.**
- **Realizar otimização de hiperparâmetros usando Grid Search com validação cruzada.**
- **Avaliar e comparar métricas dos modelos otimizados.**
- **Selecionar e salvar o melhor modelo para uso futuro.**
- **Implementar uma interface iterativa para entrada de novos dados do usuário.**


## 2. Importando as Bibliotecas Necessárias

In [17]:
# Importando as Bibliotecas Necessárias

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import joblib  # Para salvar e carregar o modelo

## 3. Carregando e Explorando o Conjunto de Dados

### 3.1 Carregando o Dataset

Para este exemplo, usaremos o conjunto de dados **Titanic** para prever a sobrevivência dos passageiros.

In [18]:
# Carregando o dataset
url = 'https://raw.githubusercontent.com/proffranciscofernando/titanic_survival_prediction/refs/heads/main/titanic.csv'
data = pd.read_csv(url)

# Visualizando as primeiras linhas
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 3.2 Análise Exploratória de Dados

Vamos verificar as informações básicas do dataset para entender sua estrutura.

In [19]:
# Informações básicas do dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### 3.3 Selecionando Recursos e Variável Alvo

Selecionaremos algumas colunas relevantes para simplificar o exemplo.

In [20]:
# Selecionando as features e a variável alvo
X = data[['Pclass', 'Sex', 'Age', 'Fare']]
y = data['Survived']

### 3.4 Tratamento de Valores Faltantes

Verificamos se existem valores faltantes nos dados selecionados.

In [21]:
# Verificando valores faltantes
X.isnull().sum()

,0
Pclass,0
Sex,0
Age,177
Fare,0


Como podemos ver, a coluna **Age** possui valores faltantes. Vamos preencher esses valores com a média da idade.

In [22]:
# Preenchendo valores faltantes na coluna 'Age' com a média
X['Age'].fillna(X['Age'].mean(), inplace=True)

<ipython-input-22-f490ce0e8f4f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Age'].fillna(X['Age'].mean(), inplace=True)


## 4. Preparação dos Dados

### 4.1 Dividindo os Dados em Treino e Teste

Dividimos o conjunto de dados em treino e teste para avaliar o desempenho dos modelos.

In [23]:
# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 5. Construindo o Pipeline

### 5.1 Criando o Pipeline de Pré-processamento

Definimos transformações para colunas numéricas e categóricas.

In [24]:
# Definindo as colunas numéricas e categóricas
numeric_features = ['Age', 'Fare']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_features = ['Pclass', 'Sex']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinando as transformações
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

## 6. Otimização de Hiperparâmetros com Grid Search

### 6.1 Definindo os Modelos e Parâmetros

Vamos definir uma lista de modelos de classificação e os hiperparâmetros que desejamos otimizar para cada um.

In [25]:
# Definindo os modelos e seus hiperparâmetros
modelos_params = {
    'Regressão Logística': {
        'modelo': LogisticRegression(max_iter=1000),
        'params': {
            'classifier__C': [0.1, 1, 10],
            'classifier__penalty': ['l2']
        }
    },
    'Random Forest': {
        'modelo': RandomForestClassifier(),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__max_depth': [None, 5, 10]
        }
    },
    'SVM': {
        'modelo': SVC(probability=True),
        'params': {
            'classifier__C': [0.1, 1, 10],
            'classifier__kernel': ['linear', 'rbf']
        }
    },
    'KNN': {
        'modelo': KNeighborsClassifier(),
        'params': {
            'classifier__n_neighbors': [3, 5, 7],
            'classifier__weights': ['uniform', 'distance']
        }
    },
    'Árvore de Decisão': {
        'modelo': DecisionTreeClassifier(),
        'params': {
            'classifier__max_depth': [None, 5, 10],
            'classifier__criterion': ['gini', 'entropy']
        }
    }
}

### 6.2 Executando o Grid Search com Validação Cruzada

Agora, vamos realizar o Grid Search com validação cruzada para cada modelo.

In [26]:
# Executando o Grid Search com validação cruzada para cada modelo
resultados = []

for nome, mp in modelos_params.items():
    modelo = mp['modelo']
    params = mp['params']

    # Criando o pipeline com o modelo atual
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', modelo)
    ])

    # Grid Search com validação cruzada
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=params,
        cv=5,
        scoring='f1',
        n_jobs=-1
    )

    # Treinando o modelo
    grid_search.fit(X_train, y_train)

    # Melhor modelo encontrado
    best_model = grid_search.best_estimator_

    # Fazendo previsões no conjunto de teste
    y_pred = best_model.predict(X_test)

    # Calculando métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Armazenando os resultados
    resultados.append({
        'Modelo': nome,
        'Melhores Hiperparâmetros': grid_search.best_params_,
        'Acurácia': acc,
        'Precisão': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Pipeline': best_model  # Armazenando o melhor modelo
    })

### 6.3 Comparando os Modelos Otimizados

Vamos visualizar as métricas de cada modelo otimizado em um dataframe.

In [27]:
# Comparando os modelos otimizados
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='F1-Score', ascending=False)
df_resultados.reset_index(drop=True, inplace=True)
df_resultados[['Modelo', 'Acurácia', 'Precisão', 'Recall', 'F1-Score', 'Melhores Hiperparâmetros']]

,Modelo,Acurácia,Precisão,Recall,F1-Score,Melhores Hiperparâmetros
0,Random Forest,0.826816,0.830769,0.729730,0.776978,"{'classifier__max_depth': 10, 'classifier__n_e..."
1,Regressão Logística,0.798883,0.779412,0.716216,0.746479,"{'classifier__C': 0.1, 'classifier__penalty': ..."
2,SVM,0.810056,0.833333,0.675676,0.746269,"{'classifier__C': 10, 'classifier__kernel': 'r..."
3,KNN,0.782123,0.777778,0.662162,0.715328,"{'classifier__n_neighbors': 3, 'classifier__we..."
4,Árvore de Decisão,0.770950,0.836735,0.554054,0.666667,"{'classifier__criterion': 'entropy', 'classifi..."


## 7. Selecionando e Salvando o Melhor Modelo

### 7.1 Selecionando o Melhor Modelo

In [28]:
# Selecionando o melhor modelo baseado no F1-Score
melhor_modelo = df_resultados.loc[0, 'Modelo']
melhor_pipeline = df_resultados.loc[0, 'Pipeline']

print(f"O melhor modelo foi: {melhor_modelo}")
print(f"Com os hiperparâmetros: {df_resultados.loc[0, 'Melhores Hiperparâmetros']}")

O melhor modelo foi: Random Forest
Com os hiperparâmetros: {'classifier__max_depth': 10, 'classifier__n_estimators': 200}


### 7.2 Salvando o Modelo

In [29]:
# Salvando o modelo usando joblib
joblib.dump(melhor_pipeline, 'melhor_modelo.pkl')

['melhor_modelo.pkl']

## 8. Carregando o Modelo e Fazendo Previsões

### 8.1 Fazendo Previsões com Novos Dados do Usuário

Nesta seção, vamos criar uma interface iterativa onde o usuário pode inserir novos dados, e o modelo fará previsões com base nesses dados.

In [30]:
# Carregando o modelo salvo
modelo_carregado = joblib.load('melhor_modelo.pkl')

#### Função para Obter Dados do Usuário

Vamos definir uma função que solicita os dados do usuário e retorna um DataFrame.

In [31]:
# Função para obter dados do usuário
def obter_dados_usuario():
    print("Insira os dados do passageiro:")

    # Solicitando input do usuário
    pclass = input("Classe (1, 2 ou 3): ")
    while pclass not in ['1', '2', '3']:
        print("Valor inválido. Por favor, insira 1, 2 ou 3.")
        pclass = input("Classe (1, 2 ou 3): ")
    pclass = int(pclass)

    sex = input("Sexo (male ou female): ").lower()
    while sex not in ['male', 'female']:
        print("Valor inválido. Por favor, insira 'male' ou 'female'.")
        sex = input("Sexo (male ou female): ").lower()

    age = input("Idade: ")
    while True:
        try:
            age = float(age)
            if age < 0:
                raise ValueError
            break
        except ValueError:
            print("Valor inválido. Por favor, insira um número positivo.")
            age = input("Idade: ")

    fare = input("Tarifa paga (entre 0 e 513 libras esterlinas): ")
    while True:
        try:
            fare = float(fare)
            if not 0 <= fare <= 513:
                raise ValueError
            break
        except ValueError:
            print("Valor inválido. Por favor, insira um número positivo.")
            fare = input("A tarifa paga deve estar entre 0 e 513 libras esterlinas: ")

    # Criando um DataFrame com os dados inseridos
    novo_passageiro = pd.DataFrame({
        'Pclass': [pclass],
        'Sex': [sex],
        'Age': [age],
        'Fare': [fare]
    })

    return novo_passageiro

#### Loop Iterativo para Previsões

Agora, vamos criar um loop que permite ao usuário inserir múltiplos conjuntos de dados.

In [32]:
# Loop iterativo para previsões
while True:
    # Obter dados do usuário
    novo_passageiro = obter_dados_usuario()

    # Fazer a previsão
    predicao = modelo_carregado.predict(novo_passageiro)
    probabilidade = modelo_carregado.predict_proba(novo_passageiro)

    resultado = "Sobreviveu" if predicao[0] == 1 else "Não Sobreviveu"
    prob_survived = probabilidade[0][1] * 100

    print(f"\nResultado da previsão: {resultado}")
    print(f"Probabilidade de sobrevivência: {prob_survived:.2f}%\n")

    # Perguntar se o usuário quer inserir outro
    continuar = input("Deseja inserir outro passageiro? (s/n): ").lower()
    if continuar != 's':
        print("Encerrando as previsões.")
        break

Insira os dados do passageiro:
Classe (1, 2 ou 3): 1
Sexo (male ou female): male
Idade: 55
Tarifa paga (entre 0 e 513 libras esterlinas): 500

Resultado da previsão: Sobreviveu
Probabilidade de sobrevivência: 57.92%

Deseja inserir outro passageiro? (s/n): n
Encerrando as previsões.


## 9. Conclusão

Neste notebook, construímos um pipeline de machine learning que inclui múltiplos modelos de classificação. Utilizamos o Grid Search com validação cruzada para otimizar os hiperparâmetros de cada modelo. Avaliamos cada modelo usando métricas como Acurácia, Precisão, Recall e F1-Score. Com base nas métricas, selecionamos o melhor modelo otimizado e o salvamos para uso futuro. Por fim, implementamos uma interface interativa que permite ao usuário inserir novos dados e obter previsões do modelo, tornando a aplicação prática e interativa.

Este processo é essencial em projetos de machine learning para garantir que estamos escolhendo o modelo e os hiperparâmetros mais adequados para o nosso problema, além de facilitar a implantação e o uso contínuo do modelo em ambientes reais.

## 10. Referências

- [Documentação do Scikit-Learn sobre Pipelines](https://scikit-learn.org/stable/modules/compose.html#pipeline)
- [Documentação do GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
- [Salvando Modelos com Joblib](https://scikit-learn.org/stable/modules/model_persistence.html)
- [Dataset Titanic no Kaggle](https://www.kaggle.com/c/titanic/data)
- [Validação Cruzada no Scikit-Learn](https://scikit-learn.org/stable/modules/cross_validation.html)
- [Métricas de Avaliação de Classificação](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)